# Simple Performance Analysis with TAU Commander and Hatchet

This notebook guides you through a simple performance analysis use case that measures time with [http://taucommander.paratools.com/](TAU Commander) and then manipulates the data using [https://github.com/LLNL/hatchet](Hatchet).

## Set up your environment

First, follow the Getting Started instructions in the [README](README.md) which will create a complete stand-alone installation of TAU Commander and Hatchet. Do not forget to add the `installation/bin` directory to your `PATH` as recommended at the end of a successful install. 

If you are working on one of the UO HPCL lab servers, you can use preinstalled tools and so can skip this step. See the [HPCL Machines](HPCL_Machines.ipynb) notebook for instructions.

## TAU Commander: Getting Started
The first thing we will learn is how to use TAU Commander to collect and organize performance analysis data. This guide is intended as a simple overview of the TAU Commander tool from Paratools http://taucommander.paratools.com/ for people unfamiliar with [TAU](https://www.cs.uoregon.edu/research/tau/home.php) or performance analysis in general. The authors of the TAU Commander provide much more comprehensive user guides.

In this tutorial, we will work with the P2Z benchmark, which you can obtain with:
```
git clone https://github.com/HPCL/p2z-tests.git
cd p2z-tests
```
Subsequent work will take place in the p2z-tests directory tree. Make sure that the installed `taucmdr/bin` directory is in your path.

## TAU Commander's Hierarchy of Performance Measurements

TAU Commander organizes performance measurement into a set of categories intended to simplify the process of setting up performance data gathering and encourage reuse of settings between experiments.

* project - top level organization
* target - hardware environment plus measurement libraries like Papi
* application - program to test; defines what libraries and measurement techniques
* measurements
* experiments
* trials

Measurements can be configured to capture different metrics (time, hardware counters) at different granularities (functions, loops, custom regions). Typically, separate measurements are produced for each thread in a multi-threaded application and each MPI task for codes based on MPI. TAU also supports GPU measurement.

## Project

The *project* is the big umbrella that keeps everything together. It exists as a .tau directory in the directory (folder) where you create your performance experiments. Typically, TAU Commander will automatically install TAU in this `.tau` subdirectory, as well as organize the measured data in folders that store standard TAU profile files, and XML/JSON files that help TAU remeber where it put everything.

To initialize a project simply type the comand below in the directory where you wish to run your performance experiments. In your `p2z-tests` directory, create an experiments subdirectory where all subsequent steps will be taking place. Note that if this is the first time you are using TAU with TAU Commander, `tau init` may take a long time since it is downloading and building multiple versions of TAU, depending on the options provided. To see a full list of options for tau init, use `tau init --help`. Here, we choose to not initialize any default target and measurement options, and also specify the use the sqlite for storing performance data (instead of the default ASCII profiles). 

```
mkdir experiments
cd experiments
tau init --backend sqlite --bare # results will be generated using sqlite format
```

![alt text](images/dash.png "TAU Commander Dashboard")

To view your current project configuration at any point, use the `tau dash` command.

## Target

The *target* defines system information including OS, architecture, compilers, and external libraries, e.g., PAPI (which is needed for hardware counter data collection). Most of this information TAU can figure out on its own, but it often needs help finding a papi installation. Also, if you want to use source instrumentation add the '--pdt' flag to let TAU know.

Initially, our goal is to do simple timing, so we will create a target without any external tools. The target name can be anything, but it helps to associate it with the machine name or type. For example, on a machine named `arya` using `gcc` compilers, we could create the target as follows:
```
tau target create arya_gcc
```

Later we can create more targets with support for PAPI counters or instrumentation (with PDT), e.g.,
```
tau target create arya_gcc_papi --papi /path/to/papi
tau target create arya_gcc_papi_pdt --papi /path/to/papi --pdt
```

## Application

### Building the application
Build the OpenMP version of the p2z benchmark with g++ (you can also build with any other C++ compiler that supports OpenMP), do the following in your `p2z-tests/experiments` directory (created earlier):
```
g++ -g -O3 -fopenmp -Dnthreads=1 -o p2z_omp_01 ../src_complete/propagate-toz-test_OMP.cpp
```
Note that p2z requires recompilation to modify the number of threads, so in the above case, the executable will run with one thread regardless of the OMP_NUM_THREADS environment variable setting. We can create other versions, e.g., one that uses 2 threads by modifying the nthreads value, e.g.:
```
g++ -g -O3 -fopenmp -Dnthreads=2 -o p2z_omp_02 ../src_complete/propagate-toz-test_OMP.cpp
```

### Creating the TAU Commander Application metadata
Application defines application specific needs such as OpenMP, MPI, TBB, etc.
Note the 'T' after --tbb is for 'True'. To see all available options, run `tau application create --help`. Here, we will create an application configuration for OpenMP:

```
tau application create p2z_omp --ompt T 
```

## Measurements

Measurements define what and how a run should record data. This includes sampling (currently recomended for this application) vs intrumentation, recording callpaths, how to store data, and most improtant what metric to measure. For TAU and our libraries to work correctly TIME must always be included and only one other metric should be measuremed at a time.

Below we have some examples of defining measurements. The --callpath 100 option is assumed by default and tells tau to record callpaths to a depth of 100.

```
tau measurement create sample_ins_cyc --sample T --callsite T --metrics TIME PAPI_TOT_INS
```

## Experiments

Within a project Experiments are the primary organizational feature. Each experiment combines an application, target, and measurement so that it has all the information to run tau on your program. As a result, tau Makefiles are stored at the experiment level. 


```
tau experiment create sampling --application p2z_omp --measurement sample_ins_cyc --target arya_papi
```

## Trials

Program runs are called *trials*, each of which is associated with the experiment that is currently selected when the trial is created. The trials in each experiment are numbered sequentially starting at 0 and the resulting data is stored as tau profiles. To create a trial, you need to specify the exact command that runs your application: `tau trial create COMMAND`. You must include all required command-line arguments in the COMMAND string.

```
tau trial create [CMD]|
tau trial create p2z_gcc_omp
```

## Workflow and Data Organization

 Once you decide to use Tau Performance Analysis for your program you can essentially follow this notebook from the top down.
 * Initialize the TAU Commander project
 * define the target system(s) and whether or not you wish to use pdt
 * define the application scenario (MPI, TBB, etc)
 * define all the measurements you want
 If you would like to go back later and add new measurements, systems or perhaps expand your program to include MPI this is a simple matter of creating new measurements, targets, OR applications
   * changes to the above definitions will not impact the others
   * Experiments on the other hand must be recreated if the underlying targets, measurements or applicaitons are altered

When creating experiments the goal is to consider what types of runs you will want to measure and limit experiments to varying one parameters per each. For example, we used the following experiments (and others) to explore the p2z benchmark.
* TODO